<a href="https://colab.research.google.com/github/sakurasakura1996/Pytorch-start-learning/blob/master/Dive_into_DL_Pytorch_5_4_5_5_%E6%B1%A0%E5%8C%96%E5%B1%82_%E5%8D%B7%E7%A7%AF%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

回忆一下，5.1节 二维卷积层里介绍的图像物体边缘检测应用中，我们构造卷积核从而精确地找到了像素变化的位置。在输入经过卷积核之后，导致相邻元素不一样，这就意味着物体边缘通过这两个元素之间。但实际图像中，我们感兴趣的物体不会总出现在固定位置：即使我们连续拍摄同一个物体也极有可能出现像素位置上的偏移。这会导致同一个边缘对应的输出可能出现在卷积输出Y中的不同位置，进而对后面的模式识别造成不便。
所以我们本节提出的池化层pooling层，他的提出就是为了缓解卷积层对位置的过度敏感性

5.4.1 二维最大池化层   和   平均池化层
同卷积层一样，池化层每次对输入数据的一个固定形状窗口（又称池化窗口）中的元素计算输出。不同于卷积层里计算输入和核的互相关性，池化层直接计算池化窗口内元素的最大值或者平均值。该运算也分别叫做最大池化或平均池化。在二维最大池化中，池化窗口从输入数组的最左上方开始按从左往右、从上往下的顺序，依次在输入数组上滑动。当池化窗口滑动到某一位置时，窗口中的输入子数组的最大值即输出数组中相应位置的元素

In [0]:
# 让我们再次回到本节开始提到的物体边缘检测的例子。现在我们将卷积层的输出作为2*2最大池化的输入。设卷积层输入X，池化层输出Y。在加入一个最大池化层之后，
# 无论 X[i,j] 和 X[i,j+1]值不同，还是X[i,j+1] 和 X[i,j+2]不同，池化层输出均有Y[i,j] =1，也就是说，使用2x2最大池化层时，只要卷积层识别的模式在
# 高和宽上移动不超过一个元素，我们依然可以将他检测出来。  下面实现池化层的前向计算实现
import torch
from torch import nn

def pool2d(X, pool_size,mode = 'max'):
    X = X.float()
    p_h, p_w = pool_size
    Y = torch.zeros(X.shape[0]- p_h +1,X.shape[1]-p_w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i,j] = X[i:i+p_h,j:j+p_w].max()
            elif mode == 'avg':
                Y[i,j] = X[i:i+p_h,j:j+p_w].mean()
    return Y

In [0]:
X = torch.tensor([[0,1,2],[3,4,5],[6,7,8]])
print(pool2d(X,(2,2)))
print(pool2d(X,(2,2),'avg'))

tensor([[4., 5.],
        [7., 8.]])
tensor([[2., 3.],
        [5., 6.]])


In [0]:
# 池化层填充和步幅
# 同卷积层一样，池化层也可以在输入的高和宽两侧的填充并调整窗口的移动步幅来改变输出形状。池化层填充和步幅与卷积层机制一样的，我们通过nn模块中的二维
# 最大池化层Maxpool2d来演示池化层填充和步幅的工作机制。我们先构造一个形状为（1，1，4，4）的输入数据，前面两个维度分别是批量和通道
X = torch.arange(16,dtype=torch.float).view(1,1,4,4)
print(X)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])


In [0]:
# 默认情况下，Maxpool2d实例里步幅和池化窗口形状不同。下面使用形状为（3，3）的池化窗口。默认获得形状为（3，3）的步幅
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [0]:
# 我们可以手动指定步幅和填充
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
print(pool2d(X))
print(pool2d(X).shape)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])
torch.Size([1, 1, 2, 2])


In [0]:
pool2d = nn.MaxPool2d((2,4), padding=(1,2),stride=(2,3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [0]:
# 5.4.3多通道
# 在处理多通道输入数据时，池化层对每个输入通道分别池化，而不是像卷积层那样将各通道的输入按通道相加。这意味着池化层的输出通道数与输入通道数相等。下面
# 将数组X 和 X+1 在通道维上连接来构造通道数为2的输入。
X = torch.cat((X,X+1),dim=1)
print(X)
print(X.shape)

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]]]])
torch.Size([1, 2, 4, 4])


In [0]:
# 池化后，我们发现输出通道仍然是2
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
print(pool2d(X))
print(pool2d(X).shape)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]]]])
torch.Size([1, 2, 2, 2])


In [0]:
# conclusion
# 池化层的一个主要作用是缓解卷积层对位置的过渡敏感性。我们可以指定池化层的填充  和  步幅，  池化层的输出通道数和输入通道数相同，他不像 1x1 卷积核，可以改变通道数

# 5.5 卷积神经网络
在3.9节（多层感知机从零开始实现）里我们构造了一个含单隐藏层的多层感知机模型来对Fashion-MNIST数据集中的图像进行分类。每张图像高和宽均为28像素
我们将图像中的像素逐行展开，得到784的向量，并输入进全连接层中。然而，这种分类方法有一定局限性
1.图像在同一列邻近的像素在这个向量中可能相距较远。他们构成的模式可能难以被模型识别
2.对于大尺寸的输入图像，使用全连接层容易造成模型过大。假设输入是高和宽均为1000像素的彩色照片（3通道）。即使全连接层输出格式仍是256，该层权重参数的形状是 3000000x256:它占用了大约3GB 的内存或显存。这带来过复杂的模型和过高的存储开销
卷积层尝试解决这两个问题。一方面，卷积层保留输入形状，使图像的像素在高和宽的两个方向上的相关性均可能被有效识别；另一方面，卷积层通过滑动窗口将同一卷积核与不同位置的输入重复计算，从而避免参数尺寸过大。
## 5.5.1 LENET模型
![image.png](attachment:image.png)
LeNet分为卷积层块和全连接层块两个部分。
卷积层块里的基本单位是卷积层后接最大池化层：卷积层用来识别图像里的空间模式，如线条和物体局部，之后的最大池化层则用来降低卷积层对位置的敏感性。卷积层块由两个这样的基本单位重复堆叠构成。在卷积层块中，每个卷积层都使用5x5的窗口，并在输出上使用sigmoid激活函数。第一个卷积层输出通道数为6，第二个卷积层输出通道数增加至16。这是因为第二个卷积层比第一个卷积层的输入的高和宽要小，所以增加输出通道使两个卷积层的参数尺寸类似。卷积层块的两个最大池化层的窗口形状均为2x2，且步幅为2.由于池化窗口与步幅形状相同，池化窗口在输入上每次滑动所覆盖的区域互不重叠
卷积层块的输出形状为（批量大小，通道，高，宽）。当卷积层快的输出传入全连接层块时，全连接层块会将小批量中每个样本变平（flatten）。也就是说，全连接层的输入形状将变成二维，其中第一维是小批量中的样本，第二维是每个样本变平后的向量表示，且向量长度为通道、高和宽的乘积。全连接层快含3个全连接层。他们的输出个数分别是120，84和10，其中10为输出的类别个数。
下面通过Sequential类来实现LeNet模型

In [0]:
import time
from torch import optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class LeNet(nn.Module):
    def __init__(self):
        super(LeNet,self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1,6,5), # in_channels,out_channels,kernel_size
            nn.Sigmoid(),
            nn.MaxPool2d(2,2),  # kernel_size, stride
            nn.Conv2d(6,16,5),
            nn.Sigmoid(),
            nn.MaxPool2d(2,2)
        )
        self.fc = nn.Sequential(
            nn.Linear(16*4*4, 120),
            nn.Sigmoid(),
            nn.Linear(120, 84),
            nn.Sigmoid(),
            nn.Linear(84, 10)
        )
    
    def forward(self, img):
        feature = self.conv(img)
        output = self.fc(feature.view(img.shape[0],-1))
        return output

In [0]:
net = LeNet()
print(net)

LeNet(
  (conv): Sequential(
    (0): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): Sigmoid()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (4): Sigmoid()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=256, out_features=120, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): Sigmoid()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [0]:
# 可以看到，在卷积层块中输入的高和宽在逐层减小。卷积层由于使用高和宽均为5的卷积核，从而将
# 高和宽分别减小为4，而池化层则将高和宽减半，但通道数则从1增加到16.全连接层则逐层减少输出个数
# 直到变成图像的类别数为10
# 5.5.2 获取数据和训练模型，人使用Fashion-MNIST数据集，然后用LeNet模型

In [0]:

import torch
import torch.nn as nn
import torchvision
import torchvision.transforms  as transforms
import sys

# 加载MNIST数据
def load_data_fashion_mnist(batch_size):
  mnist_train = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST',train=True,transform=transforms.ToTensor(),download=True)
  mnist_test = torchvision.datasets.FashionMNIST(root='~/Datasets/FashionMNIST',train=False,transform=transforms.ToTensor(),download=True)
  if sys.platform.startswith('win'):
    num_workers = 0 
  else:
    num_workers = 4
  train_iter = torch.utils.data.DataLoader(mnist_train,batch_size=batch_size, shuffle=True, num_workers=num_workers)
  test_iter = torch.utils.data.DataLoader(mnist_test,batch_size=batch_size, shuffle=False, num_workers=num_workers)
  return train_iter,test_iter

In [0]:
batch_size = 256
train_iter, test_iter = load_data_fashion_mnist(batch_size)
print(type(train_iter))

<class 'torch.utils.data.dataloader.DataLoader'>


In [0]:
# 使用GPU来运算，下面来写evaluate_accuracy函数做修改。支持GPU
def evaluate_accuracy(data_iter, net, device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')):
  acc_sum, n =0.0, 0
  with torch.no_grad():
    for X,y in data_iter:
      if isinstance(net, torch.nn.Module):
        net.eval()   # 评估模式，这回关闭dropout
        acc_sum += (net(X.to(device)).argmax(dim=1) == y.to(device)).float().sum().cpu().item()
        net.train()  # 改回训练模式
      else:
        if('is_training' in net.__code__.co_varnames):
          acc_sum += (net(X,is_training=False).argmax(dim=1) == y).float().sum().item()
        else:
          acc_sum += (net(X).argmax(dim=1) == y).float().sum().item()
      n += y.shape[0]
  return acc_sum / n


In [0]:
def train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs):
  net = net.to(device)
  print("training on ",device)
  loss = torch.nn.CrossEntropyLoss()
  batch_count = 0
  for epoch in range(num_epochs):
    train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
    for X,y in train_iter:
      X = X.to(device)
      y = y.to(device)
      y_hat = net(X)
      l = loss(y_hat,y)
      optimizer.zero_grad()
      l.backward()
      optimizer.step()
      train_l_sum += l.cpu().item()
      train_acc_sum +=(y_hat.argmax(dim=1) == y).sum().cpu().item()
      n += y.shape[0]
      batch_count +=1
    test_acc = evaluate_accuracy(test_iter, net)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f,time %.1f sec'% (epoch + 1, train_l_sum / batch_count,train_acc_sum / n, test_acc, time.time() - start))

In [0]:
lr, num_epochs = 0.001, 10
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
train_ch5(net,train_iter,test_iter,batch_size,optimizer,device,num_epochs)

training on  cuda
epoch 1, loss 0.5543, train acc 0.783, test acc 0.780,time 5.7 sec
epoch 2, loss 0.2655, train acc 0.793, test acc 0.788,time 5.7 sec
epoch 3, loss 0.1701, train acc 0.801, test acc 0.796,time 5.6 sec
epoch 4, loss 0.1227, train acc 0.809, test acc 0.803,time 5.7 sec
epoch 5, loss 0.0950, train acc 0.817, test acc 0.812,time 5.8 sec
epoch 6, loss 0.0769, train acc 0.822, test acc 0.818,time 5.8 sec
epoch 7, loss 0.0641, train acc 0.829, test acc 0.823,time 5.6 sec
epoch 8, loss 0.0545, train acc 0.835, test acc 0.825,time 5.6 sec
epoch 9, loss 0.0475, train acc 0.839, test acc 0.828,time 5.7 sec
epoch 10, loss 0.0417, train acc 0.844, test acc 0.837,time 5.5 sec
